# Libraries and Packages

In [11]:
import pandas as pd

from bertopic import BERTopic
from sklearn.feature_extraction.text import CountVectorizer

# Data

In [12]:
file_path = "/Users/catherinepollack/Documents/dartmouth/research/aim3_facebook_covid19_obesity/data"
dat_obesity = "/220120_feature_matrix_no_bert.csv"

dat_obesity = pd.read_csv(str(file_path + dat_obesity))

/Users/catherinepollack/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3165: DtypeWarning: Columns (361,363,364,366,367,375,379,384,389,394,399,400,401,402) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [13]:
dat_obesity.processed_text_bert = dat_obesity.processed_text_bert.fillna('')
topic_model = BERTopic(verbose = True, min_topic_size = 100)
topics = topic_model.fit_transform(dat_obesity.processed_text_bert)

Batches: 100%|██████████| 16328/16328 [23:04:22<00:00,  5.09s/it]     
2022-03-14 13:51:12,448 - BERTopic - Transformed documents to Embeddings
2022-03-14 15:12:27,182 - BERTopic - Reduced dimensionality with UMAP
2022-03-14 15:13:45,014 - BERTopic - Clustered UMAP embeddings with HDBSCAN


In [14]:
dat_obesity["topics"] = topics[0]

In [15]:
dat_obesity.to_csv(file_path + "/220314_obesity_with_topics.csv")
topic_model.save("220314_bert_topic_model_facebook")

/Users/catherinepollack/opt/anaconda3/lib/python3.8/site-packages/scipy/sparse/_index.py:125: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_arrayXarray(i, j, x)


In [16]:
topic_model.get_topic_info()[0:13]

,Topic,Count,Name
0,-1,251740,-1_the_to_in_is
1,0,8316,0_sexual_occurs_intercourse_sooner
2,1,8261,1_bariatric_surgery_bariatricsurgery_bariatrics
3,2,3485,2_sleep_night_deprivation_hours
4,3,3325,3_keto_carb_ketogenic_carbs
5,4,3314,4_pet_pets_dogs_dog
6,5,3063,5_covid19_virus_covid_deaths
7,6,2987,6_covid19_coronavirus_severe_factor
8,7,2811,7_read_interesting_oh_yikes
9,8,2773,8_my_myself_was_me


In [9]:
for i in range(0,12):
    print(i)
    docs = topic_model.get_representative_docs()[i]
    for d in docs:
        print(d)
        print("-----")
    print("**********")

0
#PREMATURE_EJACULATION *Premature ejaculation occurs when a man ejaculates sooner during sexual intercourse than he or his partner would like.Its when an Orgasm or Climax occurs sooner than wanted. #SYMPTOMS *Life long (primary),,,, it occurs all or nearly all of the time beginning wth ur first sexual intercourse . *Acquired (secondary). Develops after u have had previous sexual experience without ejaculatory problems #CAUSES 1•Stress & Anxiety. 2•Alcohol and Drugs 3•Diabetes. 4•High blood pressure/hypertension 5•Prostate surgery •Masturbation. 6•Depression. 7•prostate cancer & Baldness ,8•Ulcers & use of chemicals. 9•Low self-esteem. 10•Smoking & Narcotics 11•Hormone imbalance 12•Multiple sclerosis 13•Nerve Damage 14•Atherosclerosis 15•Gloom & Fear 16•Uncertainties, Sadness. 17•Fatigue and Obesity. 18•Problems in relationships. #SYMPTOMS 1.Inability to maintain an erection. 2.Decreased libido/sexual desires. 3.Early ejaculation CAUSES #Psychological Causes ☆Early sexual experience ☆